Notes:
- Predicting'Winner' instead of 'WinnerName' works a lot better which makes sense. If we want to predict future we can just map the color side (red, blue) to the name of the winner after predicting.
Important features (Take with grain of salt): 
- using both redodds and blueodds works better than an Odds difference column I made. 
- SigStrDif and AvgTDDif help too 
- WinstreakDif, WinDif, LossDif, KOOdds, and ReachDif all made the accuracy lower when I added them to my model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
ufc_data = pd.read_csv('ufc-master.csv')
ufc_data['WinnerName'] = ufc_data.apply(lambda x: x['BlueFighter'] if x['Winner'] == 'Blue' else x['RedFighter'], axis=1)
ufc_data.head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,FinishRound,FinishRoundTime,TotalFightTimeSecs,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds,WinnerName
0,Neil Magny,Carlos Prates,550.0,-800.0,550.0000,12.5000,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,1.0,4:50,290.0,1100.0,800.0,1600.0,800.0,2000.0,-400.0,Carlos Prates
1,Gerald Meerschaert,Reinier de Ridder,250.0,-310.0,250.0000,32.2581,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,1:44,704.0,700.0,200.0,600.0,165.0,1400.0,450.0,Reinier de Ridder
2,Gaston Bolanos,Cortavious Romious,205.0,-250.0,205.0000,40.0000,2024-11-09,"Las Vegas, Nevada, USA",USA,Red,...,3.0,5:00,900.0,550.0,275.0,3000.0,165.0,380.0,450.0,Gaston Bolanos
3,Luana Pinheiro,Gillian Robertson,360.0,-470.0,360.0000,21.2766,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,3.0,5:00,900.0,600.0,250.0,2200.0,110.0,900.0,500.0,Gillian Robertson
4,Mansur Abdul-Malik,Dusko Todorovic,-485.0,370.0,20.6186,370.0000,2024-11-09,"Las Vegas, Nevada, USA",USA,Red,...,1.0,2:44,164.0,600.0,NaN,800.0,NaN,-250.0,NaN,Mansur Abdul-Malik


In [3]:
def american_to_decimal(odds):
    if odds > 0:
        return odds / 100 + 1
    else:
        return 100 / abs(odds) + 1

ufc_data['red_decimal_odds'] = ufc_data['RedOdds'].apply(american_to_decimal)
ufc_data['blue_decimal_odds'] = ufc_data['BlueOdds'].apply(american_to_decimal)

ufc_data['odds_difference'] = ufc_data['red_decimal_odds'] - ufc_data['blue_decimal_odds']
ufc_data.head()

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,Winner,...,RedDecOdds,BlueDecOdds,RSubOdds,BSubOdds,RKOOdds,BKOOdds,WinnerName,red_decimal_odds,blue_decimal_odds,odds_difference
0,Neil Magny,Carlos Prates,550.0,-800.0,550.0000,12.5000,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,1100.0,800.0,1600.0,800.0,2000.0,-400.0,Carlos Prates,6.500000,1.125000,5.375000
1,Gerald Meerschaert,Reinier de Ridder,250.0,-310.0,250.0000,32.2581,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,700.0,200.0,600.0,165.0,1400.0,450.0,Reinier de Ridder,3.500000,1.322581,2.177419
2,Gaston Bolanos,Cortavious Romious,205.0,-250.0,205.0000,40.0000,2024-11-09,"Las Vegas, Nevada, USA",USA,Red,...,550.0,275.0,3000.0,165.0,380.0,450.0,Gaston Bolanos,3.050000,1.400000,1.650000
3,Luana Pinheiro,Gillian Robertson,360.0,-470.0,360.0000,21.2766,2024-11-09,"Las Vegas, Nevada, USA",USA,Blue,...,600.0,250.0,2200.0,110.0,900.0,500.0,Gillian Robertson,4.600000,1.212766,3.387234
4,Mansur Abdul-Malik,Dusko Todorovic,-485.0,370.0,20.6186,370.0000,2024-11-09,"Las Vegas, Nevada, USA",USA,Red,...,600.0,NaN,800.0,NaN,-250.0,NaN,Mansur Abdul-Malik,1.206186,4.700000,-3.493814


In [9]:
sett = ufc_data[['odds_difference','WinDif','SigStrDif','LossDif','ReachDif','AvgTDDif','KODif','SubDif', 'Winner']].dropna()
X = sett[['odds_difference','WinDif','SigStrDif','LossDif','ReachDif','AvgTDDif','KODif','SubDif']]
Y = sett['Winner'] 

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, shuffle=True) 
sett.shape

(6253, 9)

In [11]:

sett = ufc_data[['RedOdds','BlueOdds','SigStrDif','AvgTDDif', 'Winner']].dropna()
X = sett[['RedOdds','BlueOdds','SigStrDif','AvgTDDif', ]]
Y = sett['Winner'] 

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, shuffle=True) 
sett.shape

(6253, 5)

In [12]:
model = MLPClassifier(hidden_layer_sizes=(32,16,), activation='relu', max_iter=300, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 64.19%


In [7]:
# modeltree = DecisionTreeClassifier(criterion = 'entropy', random_state=42)
# modeltree.fit(X_train, y_train)

# y_pred = modeltree.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [8]:
# rf_model = RandomForestClassifier(n_estimators=150, random_state=42)
# gb_model = GradientBoostingClassifier(n_estimators=150, random_state=42)

# ensemble_model = VotingClassifier(
#     estimators=[('RandomForest', rf_model), ('GradientBoosting', gb_model)],
#     voting='soft'
# )

# ensemble_model.fit(X_train, y_train)

# y_pred = ensemble_model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# conf_matrix = confusion_matrix(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

# print(f"Accuracy: {accuracy:.2f}")
# print("Confusion Matrix:")
# print(conf_matrix)
# print("Classification Report:")
# print(class_report)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5a713994-b4e1-47cc-89fe-2b6d51a0ee77' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>